# Process Data

## LiDAR Data

In [1]:
import json
import pdal
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
import os


In [2]:
lidar_dir = "../data/test"
dsm_dir = "../data/dsm"
dtm_dir = "../data/dtm"
ndsm_dir = "../data/ndsm"

In [6]:
import os
print(os.listdir(lidar_dir))



['Philadelphia_100.las']


In [7]:
for file in os.listdir(lidar_dir):

    output_dsm = os.path.join(dsm_dir, f"{file.split('.')[0]}_dsm.tif")
    output_dtm = os.path.join(dtm_dir, f"{file.split('.')[0]}_dtm.tif")
    output_ndsm = os.path.join(ndsm_dir, f"{file.split('.')[0]}_ndsm.tif")
    
    print(output_dsm)
    print(output_dtm)
    print(output_ndsm)

    # Create a pipeline to read the LAS file and get metadata
    pipeline_json = {
        "pipeline": [
            file,
            {
                "type": "filters.info"
            }
        ]
    }

    # Run PDAL pipeline
    pipeline = pdal.Pipeline(json.dumps(pipeline_json))
    pipeline.execute()

    # Retrieve metadata
    metadata = pipeline.metadata

    nav = metadata['metadata']['filters.info']['bbox']

    max_x = nav['maxx']
    max_y = nav['maxy']
    min_x = nav['minx']
    min_y = nav['miny']

    dtm_pipeline = {
        "pipeline": [
            file,
            {
                "type": "filters.smrf",  # Simple Morphological Filter to classify ground points
                "ignore": "Classification[7:7]",  # Ignore noise
                "slope": 0.2,
                "window": 16,
                "threshold": 0.5,
                "cell": 1.0
            },
            {
                "type": "filters.range",
                "limits": "Classification[2:2]"  # Select only ground points
            },
            {
                "type": "writers.gdal",
                "filename": output_dtm,
                "output_type": "idw",  # Inverse Distance Weighting interpolation
                "resolution": 1.0,
                "bounds": f"([{min_x}, {max_x}], [{min_y}, {max_y}])"
            }
        ]
    }

    # Initialize the PDAL pipeline with the JSON string.
    p_dtm = pdal.Pipeline(json.dumps(dtm_pipeline))
    p_dtm.execute()

    dsm_pipeline = {
        "pipeline": [
            file,
            {
                "type": "writers.gdal",
                "filename": output_dsm,
                "output_type": "idw",
                "resolution": 1.0,
                "bounds": f"([{min_x}, {max_x}], [{min_y}, {max_y}])"
            }
        ]
    }

    # Execute the pipeline
    p_dsm = pdal.Pipeline(json.dumps(dsm_pipeline))
    p_dsm.execute()

    # Load the DSM and DTM data
    with rasterio.open(output_dsm) as dsm_src, rasterio.open(output_dtm) as dtm_src:
        dsm_data = dsm_src.read(1)
        dtm_data = dtm_src.read(1)

        # Compute nDSM
        ndsm_data = dsm_data - dtm_data

        # Save the result as a new raster
        ndsm_meta = dsm_src.meta.copy()
        ndsm_meta.update({"dtype": "float32"})

        with rasterio.open(output_ndsm, "w", **ndsm_meta) as dst:
            dst.write(ndsm_data.astype(np.float32), 1)

../data/dsm\Philadelphia_100_dsm.tif
../data/dtm\Philadelphia_100_dtm.tif
../data/ndsm\Philadelphia_100_ndsm.tif


RuntimeError: Unable to open stream for 'Philadelphia_100.las' with error 'Resource temporarily unavailable'